In [74]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
import pickle 

In [2]:
credit_risk=pd.read_csv('credit_risk_dataset.csv')
credit_risk.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [3]:
credit_risk.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,32581.000000,3.258100e+04,31686.000000,32581.000000,29465.000000,32581.000000,32581.000000,32581.000000
mean,27.734600,6.607485e+04,4.789686,9589.371106,11.011695,0.218164,0.170203,5.804211
std,6.348078,6.198312e+04,4.142630,6322.086646,3.240459,0.413006,0.106782,4.055001
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,23.000000,3.850000e+04,2.000000,5000.000000,7.900000,0.000000,0.090000,3.000000
50%,26.000000,5.500000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000
75%,30.000000,7.920000e+04,7.000000,12200.000000,13.470000,0.000000,0.230000,8.000000
max,144.000000,6.000000e+06,123.000000,35000.000000,23.220000,1.000000,0.830000,30.000000


In [4]:
credit_risk_copy=credit_risk.copy()

In [5]:
credit_risk.pivot_table(index='person_age',columns='loan_status',values='person_income',aggfunc='count').reset_index().sort_values(by='person_age',ascending=False)

loan_status,person_age,0,1
57,144,3.0,NaN
56,123,2.0,NaN
55,94,1.0,NaN
54,84,1.0,NaN
53,80,1.0,NaN
52,78,1.0,NaN
51,76,1.0,NaN
50,73,3.0,NaN
49,70,5.0,2.0
48,69,5.0,NaN


In [6]:
below_70=credit_risk[credit_risk['person_age']<70]
below_70.shape

(32561, 12)

In [7]:
below_70.pivot_table(index='person_emp_length',columns='loan_status',values='person_income',aggfunc='count').reset_index().sort_values(by='person_emp_length',ascending=False)

loan_status,person_emp_length,0,1
34,123.0,1.0,1.0
33,38.0,1.0,NaN
32,34.0,NaN,1.0
31,31.0,4.0,NaN
30,30.0,1.0,1.0
29,29.0,NaN,1.0
28,28.0,3.0,NaN
27,27.0,4.0,1.0
26,26.0,5.0,1.0
25,25.0,8.0,NaN


In [8]:
below_52=below_70[below_70['person_emp_length']<52]

In [9]:
below_52.shape

(31665, 12)

In [10]:
below_52.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,31665.000000,3.166500e+04,31665.000000,31665.000000,28621.000000,31665.000000,31665.000000,31665.000000
mean,27.709743,6.649461e+04,4.781462,9660.888994,11.040016,0.215474,0.169622,5.800916
std,6.132880,5.277661e+04,4.028706,6334.933174,3.229462,0.411158,0.106278,4.041164
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,23.000000,3.939600e+04,2.000000,5000.000000,7.900000,0.000000,0.090000,3.000000
50%,26.000000,5.600000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000
75%,30.000000,8.000000e+04,7.000000,12500.000000,13.480000,0.000000,0.230000,8.000000
max,69.000000,2.039784e+06,38.000000,35000.000000,23.220000,1.000000,0.830000,30.000000


In [11]:
below_52.isnull().sum()

person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length                0
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3044
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64

In [12]:
cr_data=below_52.copy()
cr_data.fillna({'loan_int_rate':cr_data['loan_int_rate'].mean()},inplace=True)
cr_data.isnull().sum()

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_status                   0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64

In [13]:
cr_data.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,31665.000000,3.166500e+04,31665.000000,31665.000000,31665.000000,31665.000000,31665.000000,31665.000000
mean,27.709743,6.649461e+04,4.781462,9660.888994,11.040016,0.215474,0.169622,5.800916
std,6.132880,5.277661e+04,4.028706,6334.933174,3.070309,0.411158,0.106278,4.041164
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,23.000000,3.939600e+04,2.000000,5000.000000,8.490000,0.000000,0.090000,3.000000
50%,26.000000,5.600000e+04,4.000000,8000.000000,11.040016,0.000000,0.150000,4.000000
75%,30.000000,8.000000e+04,7.000000,12500.000000,13.160000,0.000000,0.230000,8.000000
max,69.000000,2.039784e+06,38.000000,35000.000000,23.220000,1.000000,0.830000,30.000000


In [14]:
cr_data.groupby('loan_status').count()['person_age']

loan_status
0    24842
1     6823
Name: person_age, dtype: int64

In [15]:
cr_data.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
5,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2


In [16]:
cr_data.groupby('person_home_ownership').count()['loan_intent']

person_home_ownership
MORTGAGE    13086
OTHER         107
OWN          2410
RENT        16062
Name: loan_intent, dtype: int64

In [17]:
cr_data['loan_intent'].unique()

array(['EDUCATION', 'MEDICAL', 'VENTURE', 'PERSONAL', 'HOMEIMPROVEMENT',
       'DEBTCONSOLIDATION'], dtype=object)

In [18]:
cr_data['loan_grade'].unique()

array(['B', 'C', 'A', 'D', 'E', 'F', 'G'], dtype=object)

In [19]:
cr_data.reset_index(drop=True,inplace=True)
cr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31665 entries, 0 to 31664
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  31665 non-null  int64  
 1   person_income               31665 non-null  int64  
 2   person_home_ownership       31665 non-null  object 
 3   person_emp_length           31665 non-null  float64
 4   loan_intent                 31665 non-null  object 
 5   loan_grade                  31665 non-null  object 
 6   loan_amnt                   31665 non-null  int64  
 7   loan_int_rate               31665 non-null  float64
 8   loan_status                 31665 non-null  int64  
 9   loan_percent_income         31665 non-null  float64
 10  cb_person_default_on_file   31665 non-null  object 
 11  cb_person_cred_hist_length  31665 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 2.9+ MB


In [20]:
cr_data_cat=cr_data.copy()

In [21]:
cr_data['cb_person_default_on_file'].unique()

array(['N', 'Y'], dtype=object)

In [22]:
person_home_ownership=pd.get_dummies(cr_data_cat['person_home_ownership'],drop_first=True).astype(int)
loan_intent=pd.get_dummies(cr_data_cat['loan_intent'],drop_first=True).astype(int)
loan_grade=pd.get_dummies(cr_data_cat['loan_grade'],drop_first=True).astype(int)
cr_data_cat['cb_person_default_on_file_bin']=np.where(cr_data_cat['cb_person_default_on_file']=='Y',1,0)
cr_data_cat.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,cb_person_default_on_file_bin
0,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2,0
1,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3,0
2,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2,0
3,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4,1
4,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2,0


In [23]:
data_to_scale = cr_data_cat.drop(['person_home_ownership', 'loan_intent', 'loan_grade', 'loan_status', 'cb_person_default_on_file', 'cb_person_default_on_file_bin'],axis=1)
data_to_scale.columns

Index(['person_age', 'person_income', 'person_emp_length', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length'],
      dtype='object')

In [24]:
scaler=StandardScaler()

In [25]:
scaled_data=scaler.fit_transform(data_to_scale)
scaled_data.shape

(31665, 7)

In [26]:
scaled_df=pd.DataFrame(scaled_data,columns=['person_age', 'person_income', 'person_emp_length', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length'])
scaled_df.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length
0,-1.094078,-1.078044,0.054246,-1.367185,0.032565,-0.655105,-0.940565
1,-0.441845,-1.078044,-0.938644,-0.656827,0.596035,3.767318,-0.693107
2,-0.767962,-0.018846,-0.193977,3.999965,1.364700,3.390942,-0.940565
3,-0.604904,-0.229170,0.798914,3.999965,1.052023,3.579130,-0.445650
4,-1.094078,-1.072359,-0.690422,-1.130399,-1.270256,0.756307,-0.940565


In [27]:
scaled_df.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length
count,3.166500e+04,3.166500e+04,3.166500e+04,3.166500e+04,3.166500e+04,3.166500e+04,3.166500e+04
mean,-5.744479e-17,-1.292508e-16,1.436120e-17,2.154180e-17,1.947738e-16,-1.256605e-16,-5.744479e-17
std,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00
min,-1.257136e+00,-1.184153e+00,-1.186867e+00,-1.446114e+00,-1.830469e+00,-1.596046e+00,-9.405646e-01
25%,-7.679617e-01,-5.134668e-01,-6.904217e-01,-7.357556e-01,-8.305537e-01,-7.491988e-01,-6.931073e-01
50%,-2.787874e-01,-1.988528e-01,-1.939765e-01,-2.621835e-01,0.000000e+00,-1.846342e-01,-4.456499e-01
75%,3.734450e-01,2.559013e-01,5.506911e-01,4.481746e-01,6.904898e-01,5.681186e-01,5.441796e-01
max,6.732711e+00,3.739006e+01,8.245591e+00,3.999965e+00,3.967084e+00,6.213765e+00,5.988242e+00


In [28]:
scaled_combined=pd.concat([scaled_df,person_home_ownership,loan_intent,loan_grade,cr_data_cat['cb_person_default_on_file_bin'],cr_data_cat['loan_status']],axis=1)
scaled_combined['cb_person_default_on_file']=scaled_combined['cb_person_default_on_file_bin']
scaled_combined=scaled_combined.drop('cb_person_default_on_file_bin',axis=1)
scaled_combined.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,OTHER,OWN,RENT,...,PERSONAL,VENTURE,B,C,D,E,F,G,loan_status,cb_person_default_on_file
0,-1.094078,-1.078044,0.054246,-1.367185,0.032565,-0.655105,-0.940565,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,-0.441845,-1.078044,-0.938644,-0.656827,0.596035,3.767318,-0.693107,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,-0.767962,-0.018846,-0.193977,3.999965,1.364700,3.390942,-0.940565,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,-0.604904,-0.229170,0.798914,3.999965,1.052023,3.579130,-0.445650,0,0,1,...,0,0,0,1,0,0,0,0,1,1
4,-1.094078,-1.072359,-0.690422,-1.130399,-1.270256,0.756307,-0.940565,0,1,0,...,0,1,0,0,0,0,0,0,1,0


In [29]:
scaled_combined.groupby('loan_status').count()['OTHER']

loan_status
0    24842
1     6823
Name: OTHER, dtype: int64

In [30]:
6823/(6823+24842)

0.2154744986578241

In [31]:
target = scaled_combined['loan_status']
features=scaled_combined.drop('loan_status',axis=1)
features.shape

(31665, 22)

## Data Balancing

In [32]:
from imblearn.over_sampling import SMOTE

In [33]:
smote=SMOTE()

In [34]:
balanced_features,balanced_target=smote.fit_resample(features,target)
balanced_features.shape

(49684, 22)

In [35]:
balanced_target_df=pd.DataFrame({'target':balanced_target})
balanced_target_df.groupby('target').size()

target
0    24842
1    24842
dtype: int64

## Model Training

In [36]:
# pip install xgboost

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [38]:
x_train,x_test,y_train,y_test=train_test_split(balanced_features,balanced_target,test_size=0.20,random_state=42)

In [39]:
y_train.shape

(39747,)

In [40]:
logit=LogisticRegression()

In [41]:
logit.fit(x_train,y_train)

LogisticRegression()

In [75]:
with open('logisticPDmodel.pkl','wb') as file:
    pickle.dump(logit,file)

In [42]:
logit.score(x_train,y_train)

0.8003874506252044

In [43]:
logit_prediction=logit.predict(x_test)
logit_prediction

array([1, 0, 1, ..., 0, 1, 0])

In [44]:
print(classification_report(y_test,logit_prediction))

              precision    recall  f1-score   support

           0       0.79      0.82      0.81      4977
           1       0.82      0.78      0.80      4960

    accuracy                           0.80      9937
   macro avg       0.80      0.80      0.80      9937
weighted avg       0.80      0.80      0.80      9937



In [45]:
print(logit.coef_)

[[-0.03818107  0.06080612 -0.04202854 -0.78030443  0.79607247  1.50140728
  -0.02106066 -0.92318283 -2.21311491  0.5027356  -1.29204995 -0.37816237
  -0.66543916 -1.0693342  -1.63768414 -0.4486869  -0.60941227  1.11575899
   0.97232516  0.87558446  2.35300302 -0.05571943]]


In [46]:
imp_features_lr=pd.DataFrame({'features':balanced_features.columns,'logit_imp':logit.coef_[0]})
imp_features_lr.sort_values(by='logit_imp',ascending=False)

,features,logit_imp
20,G,2.353003
5,loan_percent_income,1.501407
17,D,1.115759
18,E,0.972325
19,F,0.875584
4,loan_int_rate,0.796072
9,RENT,0.502736
1,person_income,0.060806
6,cb_person_cred_hist_length,-0.021061
0,person_age,-0.038181


## RandomForest

In [47]:
rf=RandomForestClassifier()

In [48]:
rf.fit(x_train,y_train)

RandomForestClassifier()

In [76]:
with open('randomForestPDmodel.pkl','wb') as file:
    pickle.dump(rf,file)

In [49]:
rf.score(x_train,y_train)

0.9999748408684932

In [50]:
rf_prediction=rf.predict(x_test)
rf_prediction

array([1, 0, 0, ..., 0, 0, 0])

In [51]:
print(classification_report(y_test,rf_prediction))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      4977
           1       0.97      0.91      0.94      4960

    accuracy                           0.94      9937
   macro avg       0.94      0.94      0.94      9937
weighted avg       0.94      0.94      0.94      9937



In [52]:
rf.feature_importances_

array([0.0564642 , 0.13809131, 0.07024183, 0.07607159, 0.1496728 ,
       0.20426166, 0.05603237, 0.0003373 , 0.01979314, 0.04316292,
       0.01518928, 0.01529764, 0.0084507 , 0.01326709, 0.01754351,
       0.00758057, 0.01806275, 0.0583436 , 0.01368736, 0.00241318,
       0.00123966, 0.01479555])

In [53]:
imp_features_rf=pd.DataFrame({'features':balanced_features.columns,'rf_imp':rf.feature_importances_})
imp_features_rf.sort_values(by='rf_imp',ascending=False)

,features,rf_imp
5,loan_percent_income,0.204262
4,loan_int_rate,0.149673
1,person_income,0.138091
3,loan_amnt,0.076072
2,person_emp_length,0.070242
17,D,0.058344
0,person_age,0.056464
6,cb_person_cred_hist_length,0.056032
9,RENT,0.043163
8,OWN,0.019793


## XGBoost

In [54]:
xgb_model =XGBClassifier(tree_method='exact')

In [55]:
xgb_model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [78]:
with open('XGBPDmodel.pkl','wb') as file:
    pickle.dump(xgb_model,file)

In [56]:
xgb_model.score(x_train,y_train)

0.9693813369562483

In [57]:
xgb_prediction=xgb_model.predict(x_test)

In [58]:
print(classification_report(y_test,xgb_prediction))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      4977
           1       0.99      0.92      0.95      4960

    accuracy                           0.95      9937
   macro avg       0.96      0.95      0.95      9937
weighted avg       0.96      0.95      0.95      9937



In [59]:
xgb_model.feature_importances_

array([0.02613526, 0.03120754, 0.04295084, 0.00693278, 0.06787664,
       0.10190252, 0.06173897, 0.01024324, 0.11758587, 0.07799093,
       0.04462619, 0.05031294, 0.01881865, 0.03458407, 0.076381  ,
       0.02351616, 0.08205055, 0.071785  , 0.02122553, 0.01192732,
       0.01406067, 0.00614745], dtype=float32)

In [60]:
imp_features_xgb=pd.DataFrame({'features':balanced_features.columns,'xgb_imp':xgb_model.feature_importances_})
imp_features_xgb.sort_values(by='xgb_imp',ascending=False)

,features,xgb_imp
8,OWN,0.117586
5,loan_percent_income,0.101903
16,C,0.082051
9,RENT,0.077991
14,VENTURE,0.076381
17,D,0.071785
4,loan_int_rate,0.067877
6,cb_person_cred_hist_length,0.061739
11,HOMEIMPROVEMENT,0.050313
10,EDUCATION,0.044626


In [61]:
features_imp=pd.concat([imp_features_lr,imp_features_rf,imp_features_xgb],axis=1)
features_imp

,features,logit_imp,features,rf_imp,features,xgb_imp
0,person_age,-0.038181,person_age,0.056464,person_age,0.026135
1,person_income,0.060806,person_income,0.138091,person_income,0.031208
2,person_emp_length,-0.042029,person_emp_length,0.070242,person_emp_length,0.042951
3,loan_amnt,-0.780304,loan_amnt,0.076072,loan_amnt,0.006933
4,loan_int_rate,0.796072,loan_int_rate,0.149673,loan_int_rate,0.067877
5,loan_percent_income,1.501407,loan_percent_income,0.204262,loan_percent_income,0.101903
6,cb_person_cred_hist_length,-0.021061,cb_person_cred_hist_length,0.056032,cb_person_cred_hist_length,0.061739
7,OTHER,-0.923183,OTHER,0.000337,OTHER,0.010243
8,OWN,-2.213115,OWN,0.019793,OWN,0.117586
9,RENT,0.502736,RENT,0.043163,RENT,0.077991


In [62]:
xgb_prediction_df=pd.DataFrame({'test_indices_xgb':x_test.index,'xgb_prediction':xgb_prediction})
logit_prediction_df=pd.DataFrame({'test_indices_logit':x_test.index,'logit_prediction':logit_prediction})
rf_prediction_df=pd.DataFrame({'test_indices_rf':x_test.index,'rf_prediction':rf_prediction})

logit_prediction_df.head()

,test_indices_logit,logit_prediction
0,45873,1
1,15189,0
2,24651,1
3,31710,0
4,22193,1


In [63]:
merged_xgb=cr_data.merge(xgb_prediction_df,left_index=True,right_on='test_indices_xgb',how='left')
merged_xgb.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,test_indices_xgb,xgb_prediction
NaN,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2,0,NaN
9826.0,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3,1,1.0
NaN,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2,2,NaN
NaN,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4,3,NaN
427.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2,4,1.0


In [64]:
merged_rf=merged_xgb.merge(rf_prediction_df,left_index=True,right_on='test_indices_rf',how='left')
merged_rf.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,test_indices_xgb,xgb_prediction,test_indices_rf,rf_prediction
NaN,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2,0,NaN,NaN,NaN
8501.0,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3,1,1.0,9826.0,1.0
NaN,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2,2,NaN,NaN,NaN
NaN,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4,3,NaN,NaN,NaN
NaN,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2,4,1.0,427.0,NaN


In [65]:
merged_final=merged_rf.merge(logit_prediction_df,left_index=True,right_on='test_indices_logit',how='left')
merged_final.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,test_indices_xgb,xgb_prediction,test_indices_rf,rf_prediction,test_indices_logit,logit_prediction
NaN,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2,0,NaN,NaN,NaN,NaN,NaN
NaN,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3,1,1.0,9826.0,1.0,8501.0,NaN
NaN,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2,2,NaN,NaN,NaN,NaN,NaN
NaN,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4,3,NaN,NaN,NaN,NaN,NaN
NaN,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2,4,1.0,427.0,NaN,NaN,NaN


In [66]:
merged_final.dropna(inplace=True)

In [67]:
merged_final.shape

(277, 18)

In [68]:
final_data_wit_prediction=merged_final.drop(['test_indices_logit','test_indices_xgb','test_indices_rf'],axis=1)

In [69]:
final_data_wit_prediction


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,xgb_prediction,rf_prediction,logit_prediction
8386.0,26,56000,RENT,7.0,MEDICAL,E,22000,17.190000,1,0.39,N,3,1.0,0.0,0.0
814.0,22,19200,MORTGAGE,6.0,HOMEIMPROVEMENT,D,1400,14.910000,1,0.07,Y,4,1.0,0.0,0.0
8094.0,23,20000,MORTGAGE,7.0,EDUCATION,C,3625,11.040016,0,0.18,Y,3,0.0,0.0,0.0
3253.0,25,54000,RENT,6.0,PERSONAL,B,20000,8.880000,1,0.37,N,3,1.0,0.0,1.0
2292.0,22,24000,OWN,3.0,EDUCATION,B,2500,10.590000,0,0.10,N,2,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6815.0,36,59617,MORTGAGE,14.0,PERSONAL,D,21000,15.620000,0,0.35,N,14,0.0,0.0,0.0
7707.0,46,56000,MORTGAGE,1.0,MEDICAL,A,6000,11.040016,0,0.11,N,13,0.0,0.0,0.0
2836.0,42,38400,RENT,4.0,MEDICAL,B,13000,11.040016,1,0.34,N,17,1.0,0.0,1.0
3175.0,57,56650,RENT,1.0,PERSONAL,C,5000,15.960000,0,0.09,N,24,0.0,1.0,1.0


In [70]:
final_data_wit_prediction.to_excel('pd_prediction.xlsx',index=False)

In [71]:
print(classification_report(final_data_wit_prediction['loan_status'],final_data_wit_prediction['xgb_prediction']))

              precision    recall  f1-score   support

           0       0.92      0.99      0.96       207
           1       0.96      0.76      0.85        70

    accuracy                           0.93       277
   macro avg       0.94      0.87      0.90       277
weighted avg       0.93      0.93      0.93       277



In [72]:
from sklearn.metrics import confusion_matrix

In [73]:
confusion_matrix(final_data_wit_prediction['loan_status'],final_data_wit_prediction['xgb_prediction'])

array([[205,   2],
       [ 17,  53]])